In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# Define Xavier initialization function
def init_weights_xavier(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)  # Initialize biases to zero for consistency

# Define a single ResNet Block with Batch Normalization and Dropout
class ResNetBlock(nn.Module):
    def __init__(self, input_size):
        super(ResNetBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.fc2 = nn.Linear(input_size, input_size)
        self.bn2 = nn.BatchNorm1d(input_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # Dropout with 30% rate

    def forward(self, x):
        shortcut = x
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.bn2(self.fc2(x))
        return self.relu(x + shortcut)

# Define the complete ResNet architecture with deeper layers
class DeepResNet(nn.Module):
    def __init__(self, input_size):
        super(DeepResNet, self).__init__()
        self.fc_initial = nn.Linear(input_size, 512)
        self.block1 = ResNetBlock(512)
        self.block2 = ResNetBlock(512)
        self.block3 = ResNetBlock(512)  # Additional ResNet block
        self.block4 = ResNetBlock(512)  # Additional ResNet block
        self.fc_final = nn.Linear(512, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Dropout with 50% rate for the final layer

    def forward(self, x):
        x = self.relu(self.fc_initial(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc_final(x))
        return x

# Load and preprocess the data
def load_data(n):
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')

    # Split into train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Set batch size
batch_size = 32  # You can adjust this to any suitable size

# Train the DeepResNet model with mini-batch gradient descent and early stopping
def train_model(X_train, y_train, X_val, y_val, input_size, num_epochs=500, learning_rate=0.001, patience=10):
    # Convert data to PyTorch tensors and create DataLoader for mini-batches
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # DataLoader for mini-batch gradient descent
    
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

    # Initialize model, apply Xavier initialization, loss function, and optimizer
    model = DeepResNet(input_size)
    model.apply(init_weights_xavier)  # Apply Xavier initialization to all linear layers

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)

    # Track losses and accuracies for plotting
    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    patience_counter = 0

    # Training loop with early stopping
    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0  # To accumulate the training loss over all batches

        # Loop over mini-batches
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            train_loss = criterion(output, y_batch)
            train_loss.backward()
            optimizer.step()
            epoch_train_loss += train_loss.item()  # Accumulate training loss

        # Average training loss over all batches for the epoch
        train_losses.append(epoch_train_loss / len(train_loader))

        # Validation loss and accuracy
        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            val_loss = criterion(val_output, y_val_tensor)
            val_pred = (val_output > 0.5).float()
            val_accuracy = accuracy_score(y_val, val_pred.numpy())

            # # Early stopping logic
            # if val_loss < best_val_loss:
            #     best_val_loss = val_loss
            #     patience_counter = 0
            # else:
            #     patience_counter += 1
            #     if patience_counter >= patience:
            #         print("Early stopping triggered")
            #         break

            # Record validation loss and accuracy
            val_losses.append(val_loss.item())
            val_accuracies.append(val_accuracy)

        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, '
                  f'Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

    return model, train_losses, val_losses, val_accuracies


# Evaluate the model on the test set
def evaluate_model(model, X_test, y_test):
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    model.eval()
    with torch.no_grad():
        test_output = model(X_test_tensor)
        test_pred = (test_output > 0.5).float()
        test_accuracy = accuracy_score(y_test, test_pred.numpy())
    print(f'Test Accuracy: {test_accuracy:.4f}')
    return test_accuracy

# Plot losses and accuracies
def plot_metrics(train_losses, val_losses, val_accuracies, dataset_name):
    epochs_train = range(1, len(train_losses) + 1)
    epochs_val = range(1, len(val_losses) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot training and validation losses
    ax1.plot(epochs_train, train_losses, label="Train Loss")
    ax1.plot(epochs_val, val_losses, label="Validation Loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.set_title(f"{dataset_name} Loss over Epochs")
    ax1.legend()

    # Use the same length adjustment for validation accuracy if needed
    epochs_val_acc = range(1, len(val_accuracies) + 1)
    ax2.plot(epochs_val_acc, val_accuracies, label="Validation Accuracy", color='green')
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.set_title(f"{dataset_name} Validation Accuracy over Epochs")
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Main script
possible_n_vals = [24, 30, 45]
for n in possible_n_vals:
    print(f"\nDataset kryptonite-{n}")
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(n)
    input_size = X_train.shape[1]

    # Train and evaluate the model
    model, train_losses, val_losses, val_accuracies = train_model(X_train, y_train, X_val, y_val, input_size)
    test_accuracy = evaluate_model(model, X_test, y_test)

    # Plot metrics
    plot_metrics(train_losses, val_losses, val_accuracies, f"Dataset kryptonite-{n}")



Dataset kryptonite-24
Epoch [10/500], Train Loss: 0.6923, Val Loss: 0.6938, Val Accuracy: 0.4981
Epoch [20/500], Train Loss: 0.6924, Val Loss: 0.6949, Val Accuracy: 0.4956
Epoch [30/500], Train Loss: 0.6917, Val Loss: 0.6946, Val Accuracy: 0.5041
Epoch [40/500], Train Loss: 0.6895, Val Loss: 0.6959, Val Accuracy: 0.5049
Epoch [50/500], Train Loss: 0.6881, Val Loss: 0.6957, Val Accuracy: 0.5046
Epoch [60/500], Train Loss: 0.6848, Val Loss: 0.7022, Val Accuracy: 0.4991
Epoch [70/500], Train Loss: 0.6798, Val Loss: 0.7037, Val Accuracy: 0.5012
Epoch [80/500], Train Loss: 0.6732, Val Loss: 0.7066, Val Accuracy: 0.5019
Epoch [90/500], Train Loss: 0.6691, Val Loss: 0.7093, Val Accuracy: 0.5041
Epoch [100/500], Train Loss: 0.6606, Val Loss: 0.7113, Val Accuracy: 0.5058
Epoch [110/500], Train Loss: 0.6562, Val Loss: 0.7277, Val Accuracy: 0.4988
Epoch [120/500], Train Loss: 0.6501, Val Loss: 0.7190, Val Accuracy: 0.5010
Epoch [130/500], Train Loss: 0.6436, Val Loss: 0.7342, Val Accuracy: 0.499

KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import random

# Xavier initialization function
def init_weights_xavier(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# ResNet Block with Batch Normalization, Dropout, and Stochastic Depth
class ResNetBlock(nn.Module):
    def __init__(self, input_size, stochastic_depth_prob=0.0):
        super(ResNetBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.fc2 = nn.Linear(input_size, input_size)
        self.bn2 = nn.BatchNorm1d(input_size)
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.dropout = nn.Dropout(0.3)
        self.stochastic_depth_prob = stochastic_depth_prob

    def forward(self, x):
        if self.training and random.random() < self.stochastic_depth_prob:
            return x  # Skip this block
        else:
            shortcut = x
            x = self.leaky_relu(self.bn1(self.fc1(x)))
            x = self.dropout(x)
            x = self.bn2(self.fc2(x))
            return self.leaky_relu(x + shortcut)

# Deep ResNet architecture with additional ResNet blocks
class DeepResNet(nn.Module):
    def __init__(self, input_size, stochastic_depth_prob=0.2):
        super(DeepResNet, self).__init__()
        self.fc_initial = nn.Linear(input_size, 1024)  # Increased to 1024 units
        self.block1 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)
        self.block2 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)
        self.block3 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)
        self.block4 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)
        self.block5 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)  # Additional block
        self.block6 = ResNetBlock(1024, stochastic_depth_prob=stochastic_depth_prob)  # Additional block
        self.fc_final = nn.Linear(1024, 1)
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.fc_initial(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.dropout(x)
        x = self.fc_final(x)  # No sigmoid here since BCEWithLogitsLoss includes it
        return x

# Load and preprocess data
def load_data(n):
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Set batch size
batch_size = 32

# Train the DeepResNet model with mini-batch gradient descent and early stopping
def train_model(X_train, y_train, X_val, y_val, input_size, num_epochs=500, learning_rate=0.001, patience=10):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

    model = DeepResNet(input_size)
    model.apply(init_weights_xavier)

    criterion = nn.BCEWithLogitsLoss()  # Updated to BCEWithLogitsLoss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)  # Reduced weight decay

    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            train_loss = criterion(output, y_batch)
            train_loss.backward()
            optimizer.step()
            epoch_train_loss += train_loss.item()

        train_losses.append(epoch_train_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            val_loss = criterion(val_output, y_val_tensor)
            val_pred = (torch.sigmoid(val_output) > 0.5).float()  # Apply sigmoid here for accuracy calculation
            val_accuracy = accuracy_score(y_val, val_pred.numpy())

            val_losses.append(val_loss.item())
            val_accuracies.append(val_accuracy)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, '
                  f'Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

    return model, train_losses, val_losses, val_accuracies

# Evaluate model on test set
def evaluate_model(model, X_test, y_test):
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    model.eval()
    with torch.no_grad():
        test_output = model(X_test_tensor)
        test_pred = (torch.sigmoid(test_output) > 0.5).float()  # Apply sigmoid for final output thresholding
        test_accuracy = accuracy_score(y_test, test_pred.numpy())
    print(f'Test Accuracy: {test_accuracy:.4f}')
    return test_accuracy

# Plot losses and accuracies
def plot_metrics(train_losses, val_losses, val_accuracies, dataset_name):
    epochs_train = range(1, len(train_losses) + 1)
    epochs_val = range(1, len(val_losses) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.plot(epochs_train, train_losses, label="Train Loss")
    ax1.plot(epochs_val, val_losses, label="Validation Loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.set_title(f"{dataset_name} Loss over Epochs")
    ax1.legend()

    epochs_val_acc = range(1, len(val_accuracies) + 1)
    ax2.plot(epochs_val_acc, val_accuracies, label="Validation Accuracy", color='green')
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.set_title(f"{dataset_name} Validation Accuracy over Epochs")
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Main script
possible_n_vals = [24, 30, 45]
for n in possible_n_vals:
    print(f"\nDataset kryptonite-{n}")
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(n)
    input_size = X_train.shape[1]

    model, train_losses, val_losses, val_accuracies = train_model(X_train, y_train, X_val, y_val, input_size)
    test_accuracy = evaluate_model(model, X_test, y_test)

    plot_metrics(train_losses, val_losses, val_accuracies, f"Dataset kryptonite-{n}")



Dataset kryptonite-24
Epoch [10/500], Train Loss: 0.7168, Val Loss: 0.7314, Val Accuracy: 0.5009
Epoch [20/500], Train Loss: 0.7033, Val Loss: 0.7229, Val Accuracy: 0.5006
Epoch [30/500], Train Loss: 0.6937, Val Loss: 0.6987, Val Accuracy: 0.5000
Epoch [40/500], Train Loss: 0.6864, Val Loss: 0.7029, Val Accuracy: 0.5055
Epoch [50/500], Train Loss: 0.6712, Val Loss: 0.7094, Val Accuracy: 0.5017
Epoch [60/500], Train Loss: 0.6369, Val Loss: 0.7596, Val Accuracy: 0.4992
Epoch [70/500], Train Loss: 0.6013, Val Loss: 0.8230, Val Accuracy: 0.4978
Epoch [80/500], Train Loss: 0.5394, Val Loss: 0.9284, Val Accuracy: 0.4978
Epoch [90/500], Train Loss: 0.4854, Val Loss: 0.9892, Val Accuracy: 0.4996
Epoch [100/500], Train Loss: 0.4419, Val Loss: 1.0999, Val Accuracy: 0.5009
Epoch [110/500], Train Loss: 0.4003, Val Loss: 1.1068, Val Accuracy: 0.4962
Epoch [120/500], Train Loss: 0.3711, Val Loss: 1.2260, Val Accuracy: 0.4958
Epoch [130/500], Train Loss: 0.3411, Val Loss: 1.3688, Val Accuracy: 0.497

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import random

# Xavier initialization function
def init_weights_xavier(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# ResNet Block with Batch Normalization, Dropout, and Stochastic Depth
class ResNetBlock(nn.Module):
    def __init__(self, input_size, stochastic_depth_prob=0.0):
        super(ResNetBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.fc2 = nn.Linear(input_size, input_size)
        self.bn2 = nn.BatchNorm1d(input_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.stochastic_depth_prob = stochastic_depth_prob

    def forward(self, x):
        if self.training and random.random() < self.stochastic_depth_prob:
            return x  # Skip this block
        else:
            shortcut = x
            x = self.relu(self.bn1(self.fc1(x)))
            x = self.dropout(x)
            x = self.bn2(self.fc2(x))
            return self.relu(x + shortcut)

# Deep ResNet architecture with additional ResNet blocks
class DeepResNet(nn.Module):
    def __init__(self, input_size, stochastic_depth_prob=0.2):
        super(DeepResNet, self).__init__()
        self.fc_initial = nn.Linear(input_size, 512)
        self.block1 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)
        self.block2 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)
        self.block3 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)
        self.block4 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)
        self.block5 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)  # Additional block
        self.block6 = ResNetBlock(512, stochastic_depth_prob=stochastic_depth_prob)  # Additional block
        self.fc_final = nn.Linear(512, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc_initial(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc_final(x))
        return x

# Load and preprocess data
def load_data(n):
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Set batch size
batch_size = 32

# Train the DeepResNet model with mini-batch gradient descent and early stopping
def train_model(X_train, y_train, X_val, y_val, input_size, num_epochs=500, learning_rate=0.001, patience=10):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

    model = DeepResNet(input_size)
    model.apply(init_weights_xavier)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)

    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            train_loss = criterion(output, y_batch)
            train_loss.backward()
            optimizer.step()
            epoch_train_loss += train_loss.item()

        train_losses.append(epoch_train_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            val_loss = criterion(val_output, y_val_tensor)
            val_pred = (val_output > 0.5).float()
            val_accuracy = accuracy_score(y_val, val_pred.numpy())

            val_losses.append(val_loss.item())
            val_accuracies.append(val_accuracy)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, '
                  f'Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

    return model, train_losses, val_losses, val_accuracies

# Evaluate model on test set
def evaluate_model(model, X_test, y_test):
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    model.eval()
    with torch.no_grad():
        test_output = model(X_test_tensor)
        test_pred = (test_output > 0.5).float()
        test_accuracy = accuracy_score(y_test, test_pred.numpy())
    print(f'Test Accuracy: {test_accuracy:.4f}')
    return test_accuracy

# Plot losses and accuracies
def plot_metrics(train_losses, val_losses, val_accuracies, dataset_name):
    epochs_train = range(1, len(train_losses) + 1)
    epochs_val = range(1, len(val_losses) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.plot(epochs_train, train_losses, label="Train Loss")
    ax1.plot(epochs_val, val_losses, label="Validation Loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.set_title(f"{dataset_name} Loss over Epochs")
    ax1.legend()

    epochs_val_acc = range(1, len(val_accuracies) + 1)
    ax2.plot(epochs_val_acc, val_accuracies, label="Validation Accuracy", color='green')
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.set_title(f"{dataset_name} Validation Accuracy over Epochs")
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Main script
possible_n_vals = [24, 30, 45]
for n in possible_n_vals:
    print(f"\nDataset kryptonite-{n}")
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(n)
    input_size = X_train.shape[1]

    model, train_losses, val_losses, val_accuracies = train_model(X_train, y_train, X_val, y_val, input_size)
    test_accuracy = evaluate_model(model, X_test, y_test)

    plot_metrics(train_losses, val_losses, val_accuracies, f"Dataset kryptonite-{n}")



Dataset kryptonite-24
Epoch [10/500], Train Loss: 0.6935, Val Loss: 0.6950, Val Accuracy: 0.4999
Epoch [20/500], Train Loss: 0.6928, Val Loss: 0.6945, Val Accuracy: 0.5038
Epoch [30/500], Train Loss: 0.6923, Val Loss: 0.6941, Val Accuracy: 0.4994
Epoch [40/500], Train Loss: 0.6907, Val Loss: 0.6944, Val Accuracy: 0.5033
Epoch [50/500], Train Loss: 0.6907, Val Loss: 0.6949, Val Accuracy: 0.5047
Epoch [60/500], Train Loss: 0.6900, Val Loss: 0.6960, Val Accuracy: 0.4967
Epoch [70/500], Train Loss: 0.6886, Val Loss: 0.6973, Val Accuracy: 0.4972
Epoch [80/500], Train Loss: 0.6878, Val Loss: 0.6975, Val Accuracy: 0.4933
Epoch [90/500], Train Loss: 0.6859, Val Loss: 0.7001, Val Accuracy: 0.4992
Epoch [100/500], Train Loss: 0.6837, Val Loss: 0.6993, Val Accuracy: 0.5039
Epoch [110/500], Train Loss: 0.6836, Val Loss: 0.6990, Val Accuracy: 0.5002
Epoch [120/500], Train Loss: 0.6812, Val Loss: 0.7035, Val Accuracy: 0.5003
Epoch [130/500], Train Loss: 0.6800, Val Loss: 0.7068, Val Accuracy: 0.502

KeyboardInterrupt: 